# Model Exercises

*This tutorial is mainly based on Laurence Moroney's excellent video tutorial [Basic Computer Vision with ML](https://www.youtube.com/watch?v=bemDFpNooA8).*

Let's take a look at a scenario where we can recognize different items of clothing, trained from a dataset containing 10 different types.

Let's start with our import of TensorFlow and Keras

In [7]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

2.7.1


### Data

The Fashion MNIST data is available directly in the tf.keras datasets API. You load it like this:

In [8]:
mnist = keras.datasets.fashion_mnist

Calling *load_data* on this object will give you two sets of two lists, these will be the training and testing values for the graphics that contain the clothing items and their labels.


In [9]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

In [4]:
training_images  = training_images / 255.0
test_images = test_images / 255.0

### Model

#### Define model

Let's now design the model. There's quite a few new concepts here, but don't worry, you'll get the hang of them. 

In [6]:
model = tf.keras.models.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28, 28)), 
                            tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.Dropout(0.2), 
                            tf.keras.layers.Dense(10)
])

2022-03-10 09:05:18.887294: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Sequential**: That defines a SEQUENCE of layers in the neural network

**Flatten**: Remember earlier where our images were a square, when you printed them out? Flatten just takes that square and turns it into a 1 dimensional set.

**Dense**: Adds a layer of neurons

Each layer of neurons need an **activation function** to tell them what to do. There's lots of options, but just use these for now. 

**Relu** effectively means "If X>0 return X, else return 0" -- so what it does it it only passes values 0 or greater to the next layer in the network.

**Softmax** takes a set of values, and effectively picks the biggest one, so, for example, if the output of the last layer looks like [0.1, 0.1, 0.05, 0.1, 9.5, 0.1, 0.05, 0.05, 0.05], it saves you from fishing through it looking for the biggest value, and turns it into [0,0,0,0,1,0,0,0,0] -- The goal is to save a lot of coding!


#### Build model

The next thing to do, now the model is defined, is to actually build it. You do this by compiling it with an optimizer and loss function as before -- and then you train it by calling **model.fit** asking it to fit your training data to your training labels -- i.e. have it figure out the relationship between the training data and its actual labels, so in future if you have data that looks like the training data, then it can make a prediction for what that data would look like. 

In [31]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4954 - accuracy: 0.8248
Epoch 2/5
1875/1875 [==============================] - 2s 990us/step - loss: 0.3690 - accuracy: 0.8655
Epoch 3/5
1875/1875 [==============================] - 2s 980us/step - loss: 0.3337 - accuracy: 0.8781
Epoch 4/5
1875/1875 [==============================] - 2s 947us/step - loss: 0.3102 - accuracy: 0.8859
Epoch 5/5
1875/1875 [==============================] - 2s 936us/step - loss: 0.2932 - accuracy: 0.8921


#### Evaluate model

Once it's done training -- you should see an accuracy value at the end of the final epoch. It might look something like 0.9098. This tells you that your neural network is about 91% accurate in classifying the training data. I.E., it figured out a pattern match between the image and the labels that worked 91% of the time. Not great, but not bad considering it was only trained for 5 epochs and done quite quickly.

But how would it work with unseen data? That's why we have the test images. We can call ``model.evaluate``, and pass in the two sets, and it will report back the loss for each. Let's give it a try:

In [32]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 597us/step - loss: 0.3618 - accuracy: 0.8677


[0.3617934584617615, 0.8676999807357788]

For me, that returned a accuracy of about .8838, which means it was about 88% accurate. As expected it probably would not do as well with *unseen* data as it did with data it was trained on!  As you go through this course, you'll look at ways to improve this. 

To explore further, try the below exercises:


## Exercises

### Exercise 1

For this first exercise run the below code: It creates a set of classifications for each of the test images, and then prints the first entry in the classifications. The output, after you run it is a list of numbers. Why do you think this is, and what do those numbers represent? 

In [33]:
classifications = model.predict(test_images)

In [34]:

print(*classifications[0], sep=' , ')

3.8688745e-06 , 4.0038034e-07 , 2.0577363e-06 , 2.3397274e-07 , 1.5762147e-07 , 0.0072907703 , 1.0248237e-05 , 0.052735463 , 2.207717e-06 , 0.93995464


Hint: try running print(test_labels[0]) -- and you'll get a 9. Does that help you understand why this list looks the way it does? 

In [35]:
print(test_labels[0])

9


A: What does this list represent?

1.   It's 10 random meaningless values
2.   It's the first 10 classifications that the computer made
3.   It's the probability that this item is each of the 10 classes

Answer: The correct answer is (3). The output of the model is a list of 10 numbers. These numbers are a probability that the value being classified is the corresponding value, i.e. the first value in the list is the probability that the handwriting is of a '0', the next is a '1' etc. Notice that they are all VERY LOW probabilities.

For the 7, the probability was .999+, i.e. the neural network is telling us that it's almost certainly a 7.

B: How do you know that this list tells you that the item is an ankle boot?

1.   There's not enough information to answer that question
1.   The 10th element on the list is the biggest, and the ankle boot is labelled 9
1.   The ankle boot is label 9, and there are 0->9 elements in the list




Answer: The correct answer is (2). Both the list and the labels are 0 based, so the ankle boot having label 9 means that it is the 10th of the 10 classes. The list having the 10th element being the highest value means that the Neural Network has predicted that the item it is classifying is most likely an ankle boot

### Exercise 2

Let's now look at the layers in your model. Experiment with different values for the dense layer with 512 neurons. What different results do you get for loss, training time etc? Why do you think that's the case? 

In [36]:
import tensorflow as tf

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(1024, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4681
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3594
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3198
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2946
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3469
[2.3763712e-05 1.1087228e-06 1.0643064e-07 4.1774744e-08 1.0095393e-06
 2.8134778e-01 1.3462964e-05 8.3642833e-02 1.0747694e-05 6.3495916e-01]
9


Question 1. Increase to 1024 Neurons -- What's the impact?

1. Training takes longer, but is more accurate
2. Training takes longer, but no impact on accuracy
3. Training takes the same time, but is more accurate

Answer: The correct answer is (1) by adding more Neurons we have to do more calculations, slowing down the process, but in this case they have a good impact -- we do get more accurate. That doesn't mean it's always a case of 'more is better', you can hit the law of diminishing returns very quickly!

### Exercise 3

What would happen if you remove the Flatten() layer. Why do you think that's the case? 

You get an error about the shape of the data. It may seem vague right now, but it reinforces the rule of thumb that the first layer in your network should be the same shape as your data. Right now our data is 28x28 images, and 28 layers of 28 neurons would be infeasible, so it makes more sense to 'flatten' that 28,28 into a 784x1. Instead of wriitng all the code to handle that ourselves, we add the Flatten() layer at the begining, and when the arrays are loaded into the model later, they'll automatically be flattened for us.

In [37]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0


model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

# This version has the 'flatten' removed. Replace the above with this one to see the error.
#model = tf.keras.models.Sequential([tf.keras.layers.Dense(64, activation=tf.nn.relu),
#                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])


model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

2.7.1
Epoch 1/5
1875/1875 [==============================] - 1s 609us/step - loss: 0.5227
Epoch 2/5
1875/1875 [==============================] - 1s 621us/step - loss: 0.3946
Epoch 3/5
1875/1875 [==============================] - 1s 607us/step - loss: 0.3567
Epoch 4/5
1875/1875 [==============================] - 1s 608us/step - loss: 0.3320
Epoch 5/5
313/313 [==============================] - 0s 497us/step - loss: 0.3664
[2.2379854e-05 2.8669564e-07 2.8954889e-07 7.0098430e-08 1.2880288e-06
 3.8433965e-02 4.8957368e-06 3.1837586e-02 4.2607207e-04 9.2927325e-01]
9


### Exercise 4

Consider the final (output) layers. Why are there 10 of them? What would happen if you had a different amount than 10? For example, try training the network with 5

You get an error as soon as it finds an unexpected value. Another rule of thumb -- the number of neurons in the last layer should match the number of classes you are classifying for. In this case it's the digits 0-9, so there are 10 of them, hence you should have 10 neurons in your final layer.

In [38]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

# Replace the above model definiton with this one to see the network with 5 output layers
# And you'll see errors as a result!
# model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
#                                    tf.keras.layers.Dense(64, activation=tf.nn.relu),
#                                    tf.keras.layers.Dense(5, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

2.7.1
Epoch 1/5
1875/1875 [==============================] - 2s 814us/step - loss: 0.5183
Epoch 2/5
1875/1875 [==============================] - 2s 838us/step - loss: 0.3906
Epoch 3/5
1875/1875 [==============================] - 2s 848us/step - loss: 0.3565
Epoch 4/5
1875/1875 [==============================] - 2s 907us/step - loss: 0.3291
Epoch 5/5
313/313 [==============================] - 0s 508us/step - loss: 0.3488
[2.8979478e-06 4.2406658e-11 1.5246025e-06 6.8555011e-08 2.4492019e-06
 4.0203530e-02 1.3005871e-05 1.3145020e-02 1.0377702e-05 9.4662112e-01]
9


### Exercise 5

Consider the effects of additional layers in the network. What will happen if you add another layer between the one with 512 and the final layer with 10. 

Ans: There isn't a significant impact -- because this is relatively simple data. For far more complex data (including color images to be classified as flowers that you'll see in the next lesson), extra layers are often necessary. 

In [39]:
import tensorflow as tf
print(tf.__version__)

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(256, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=5)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[0])
print(test_labels[0])

2.7.1
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4676
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3543
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3176
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2941
Epoch 5/5
313/313 [==============================] - 1s 821us/step - loss: 0.3372
[1.12215595e-07 2.23647376e-07 6.96224767e-09 2.85428463e-08
 1.25491360e-08 1.32345688e-03 1.07479963e-07 5.07882796e-02
 1.15422836e-07 9.47887719e-01]
9


### Exercise 6

Consider the impact of training for more or less epochs. Why do you think that would be the case? 

- Try 15 epochs -- you'll probably get a model with a much better loss than the one with 5
- Try 30 epochs -- you might see the loss value stops decreasing, and sometimes increases. 

This is a side effect of something called 'overfitting' and it's something you need to keep an eye out for when training neural networks. There's no point in wasting your time training if you aren't improving your loss.

In [40]:
import tensorflow as tf

mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels) ,  (test_images, test_labels) = mnist.load_data()

training_images = training_images/255.0
test_images = test_images/255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy')

model.fit(training_images, training_labels, epochs=30)

model.evaluate(test_images, test_labels)

classifications = model.predict(test_images)

print(classifications[34])
print(test_labels[34])

Epoch 1/30
1875/1875 [==============================] - 1s 678us/step - loss: 0.4939
Epoch 2/30
1875/1875 [==============================] - 1s 658us/step - loss: 0.3737
Epoch 3/30
1875/1875 [==============================] - 1s 665us/step - loss: 0.3369
Epoch 4/30
1875/1875 [==============================] - 1s 680us/step - loss: 0.3127
Epoch 5/30
1875/1875 [==============================] - 1s 720us/step - loss: 0.2952
Epoch 6/30
1875/1875 [==============================] - 1s 722us/step - loss: 0.2811
Epoch 7/30
1875/1875 [==============================] - 1s 705us/step - loss: 0.2692
Epoch 8/30
1875/1875 [==============================] - 1s 724us/step - loss: 0.2562
Epoch 9/30
1875/1875 [==============================] - 1s 703us/step - loss: 0.2495
Epoch 10/30
1875/1875 [==============================] - 1s 641us/step - loss: 0.2399
Epoch 11/30
1875/1875 [==============================] - 1s 655us/step - loss: 0.2313
Epoch 12/30
1875/1875 [==============================] - 1s 634

### Exercise 7

Before you trained, you normalized the data, going from values that were 0-255 to values that were 0-1. What would be the impact of removing that? Here's the complete code to give it a try. Why do you think you get different results? 

In [41]:
import tensorflow as tf

mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
# To experiment with removing normalization, comment out the following 2 lines
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(training_images, training_labels, epochs=5)
model.evaluate(test_images, test_labels)
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4741
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3601
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3211
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2983
Epoch 5/5
313/313 [==============================] - 0s 904us/step - loss: 0.3353
[6.1326890e-09 8.7253245e-09 1.7684078e-10 7.6400281e-10 9.1014730e-11
 6.3834351e-04 7.5498754e-08 4.2376164e-02 6.0282437e-09 9.5698541e-01]
9


### Exercise 8

Earlier when you trained for extra epochs you had an issue where your loss might change. It might have taken a bit of time for you to wait for the training to do that, and you might have thought 'wouldn't it be nice if I could stop the training when I reach a desired value?' -- i.e. 95% accuracy might be enough for you, and if you reach that after 3 epochs, why sit around waiting for it to finish a lot more epochs....So how would you fix that? Like any other program...you have callbacks! Let's see them in action...

In [42]:
import tensorflow as tf
print(tf.__version__)

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images/255.0
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])

2.7.1
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4752 - accuracy: 0.8303
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3581 - accuracy: 0.8686
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3219 - accuracy: 0.8814
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3000 - accuracy: 0.8889
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2806 - accuracy: 0.8941
